# Trendyol Data Science Bootcamp - You Do #2

## Import ve datayı hazırlama

In [135]:
import pandas as pd
import numpy as np
from tqdm import trange

In [136]:
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
                 names=['user_id', 'item_id', 'rating', 'timestamp'])
r = df.pivot(index='user_id', columns='item_id', values='rating').values

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [137]:
# r = np.array([[7, 6, 7, 4, 5, 4],
#               [6, 7, np.nan, 4, 3, 4],
#               [np.nan, 3, 3, 1, 1, np.nan],
#               [1, 2, 3, 3, 3, 4],
#               [1, np.nan, 1, 2, 3, 3]])

In [138]:
r

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [139]:
irow, jcol = np.where(~np.isnan(r))

idx = np.random.choice(np.arange(100_000), 1000, replace=False)
# idx = np.random.choice(np.arange(5), 3, replace=False)

test_irow = irow[idx]
test_jcol = jcol[idx]

In [140]:
r_train = r.copy()
r_train[test_irow, test_jcol] = np.nan

## Part 1 - Gradient Descent

In [141]:
def gradient_descent(r: np.array, lr=0.001, max_iteration=500, early_stopping=-1):
  m, n = r.shape
  b_user, b_item = np.random.rand(m), np.random.rand(n)
  
  loss_list = [float("inf")]

  t = trange(max_iteration)
  for it in t:
    loss = 0
    grad_b_user, grad_b_item = np.zeros(m), np.zeros(n)

    for i in range(r.shape[0]):
      for j in range(r.shape[1]):
        if np.isnan(r[i, j]):
          continue
        y = r[i, j]
        y_hat = b_user[i] + b_item[j]

        grad_b_user[i] += (y_hat - y)
        grad_b_item[j] += (y_hat - y)

        loss +=  ((y - y_hat) ** 2) / 2

    b_user -= lr * grad_b_user
    b_item -= lr * grad_b_item

    if early_stopping != -1 and loss-loss_list[-1] > -early_stopping:
            print(f"I do early stoping at iteration {it}")
            break

    t.set_description(f'Previous loss: {loss_list[-1]:.2f}, CURRENT LOSS: {loss:.2f}')
    loss_list.append(loss)

  loss_list = loss_list[1:]
  return b_user, b_item, loss_list

In [90]:
# b_user, b_item, loss_list = gradient_descent(r_train, lr=0.001, max_iteration=150, early_stopping=0.5)

Previous loss: 46.10	CURRENT LOSS: 45.60:  34%|███▍      | 51/150 [00:00<00:00, 584.30it/s]

I do early stoping at iteration 51


In [ ]:
# r_hat = np.zeros(r.shape)

# for u in range(r.shape[0]):
#       for j in range(r.shape[1]):
#         r_hat[u, j] = b_user[u] + b_item[j]

In [ ]:
# r

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [ ]:
# np.around(r_hat, 2)

array([[3.96, 3.3 , 3.12, ..., 2.16, 2.06, 2.6 ],
       [3.96, 3.3 , 3.12, ..., 2.16, 2.06, 2.59],
       [3.33, 2.68, 2.49, ..., 1.53, 1.43, 1.97],
       ...,
       [4.03, 3.37, 3.19, ..., 2.23, 2.13, 2.67],
       [4.36, 3.71, 3.52, ..., 2.56, 2.46, 3.  ],
       [3.85, 3.2 , 3.01, ..., 2.06, 1.96, 2.49]])

## Part 2 - Lambda'lı loss function içeren GD

### Fonksiyon üzerinde geliştirmeler

In [146]:
def gradient_descent2(r: np.array, lr=0.001, max_iteration=500, early_stopping=-1, lam=0.01, verbose=True, leave_bar=True):
  m, n = r.shape
  b_user, b_item = np.random.rand(m), np.random.rand(n)
  
  loss_list = [float("inf")]

  t = trange(max_iteration, leave=leave_bar)
  for it in t:
    loss = 0
    grad_b_user, grad_b_item = np.zeros(m), np.zeros(n)

    for i in range(r.shape[0]):
      for j in range(r.shape[1]):
        if np.isnan(r[i, j]):
          continue
        y = r[i, j]
        y_hat = b_user[i] + b_item[j]

        grad_b_user[i] += (y_hat - y)
        grad_b_item[j] += (y_hat - y)

        loss +=  ((y - y_hat) ** 2) / 2

    grad_b_user += lam * b_user
    grad_b_item += lam * b_item    
    loss += (lam/2) * (np.sum(b_user ** 2) + np.sum(b_item ** 2))

    b_user -= lr * grad_b_user
    b_item -= lr * grad_b_item

    if early_stopping != -1 and loss-loss_list[-1] > -early_stopping:
      print(f"\nI do early stoping at iteration {it}.")
      break

    if verbose:
      t.set_description(f'Previous loss: {loss_list[-1]:.2f}, CURRENT LOSS: {loss:.2f}')
    
    loss_list.append(loss)
  
  loss_list = loss_list[1:]
  return b_user, b_item, loss_list

In [ ]:
# b_user, b_item, loss_list = gradient_descent2(r_train, lr=0.001, max_iteration=150, early_stopping=0.5, lam=0.02)

Part 1'deki GD fonksiyonumuz aslında Part 2'dekinin lambda=0 hiperparametreli özel hali olduğundan; yeni fonksiyonumuz ilk durum için de kullanılabilir. 

Aşağıdaki 2 fonksiyon çağrısı aynıdır:

```
gradient_descent(r)
gradient_descent2(r, lam=0)

```

### Test datası ile optimizasyon

In [ ]:
lambdas = np.linspace(0, 1, 11)
lambdas = np.around(lambdas, 4)
lambda_losses = []

for lam in lambdas:
  print(f"Performing GD for lambda = {lam} ...")
  b_user, b_item, loss_list = gradient_descent2(r_train, lr=0.001, max_iteration=100, early_stopping=0.5, lam=lam, leave_bar=False)
  test_loss = 0

  for i, j in zip(test_irow, test_jcol):
    y = r[i, j]
    y_hat = b_user[i] + b_item[j]
    test_loss +=  ((y - y_hat) ** 2) / 2

  test_loss += (lam/2) * (np.sum(b_user ** 2) + np.sum(b_item ** 2))
  print(f"Test loss = {test_loss}.\n")
  lambda_losses.append(test_loss)



lambda_loss_analysis = dict(zip(lambdas, lambda_losses))

for k, v in lambda_loss_analysis.items():
  print("Lambda:", k, "\tLoss:", round(v, 4))

Performing GD for lambda = 0.0 ...


Previous loss: 63374.25, CURRENT LOSS: 60592.94:   9%|▉         | 9/100 [00:29<04:59,  3.29s/it]